In [1]:
import torch, time
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device() # GPU device on a hardware has an ID 

0

In [4]:
torch.cuda.get_device_name(0) # Get device name using the ID

'NVIDIA GeForce MX130'

In [5]:
torch.cuda.memory_allocated() # Current GPU memory usage by tensors (in bytes)

0

In [6]:
torch.cuda.memory_reserved() # Current GPU memory managed by caching allocator (in bytes)

0

In [7]:
x = torch.FloatTensor([1.0,2.0,3.0])
x

tensor([1., 2., 3.])

In [8]:
x.device

device(type='cpu')

In [9]:
x = torch.FloatTensor([1.0,2.0,3.0]).cuda()
x

tensor([1., 2., 3.], device='cuda:0')

In [10]:
x.device

device(type='cuda', index=0)

In [11]:
torch.cuda.memory_allocated()

512

In [12]:
class Model(nn.Module):

    def __init__(self,in_features=4,h1=17,h2=18,out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.out = nn.Linear(h2,out_features)

    def forward(self,X):
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        return self.out(X)

In [13]:
torch.manual_seed(101)
model = Model()
model

Model(
  (fc1): Linear(in_features=4, out_features=17, bias=True)
  (fc2): Linear(in_features=17, out_features=18, bias=True)
  (out): Linear(in_features=18, out_features=3, bias=True)
)

In [14]:
next(model.parameters()).is_cuda

False

In [15]:
gpumodel = model.cuda()
gpumodel

Model(
  (fc1): Linear(in_features=4, out_features=17, bias=True)
  (fc2): Linear(in_features=17, out_features=18, bias=True)
  (out): Linear(in_features=18, out_features=3, bias=True)
)

In [16]:
torch.cuda.memory_allocated()

4608

In [17]:
next(gpumodel.parameters()).is_cuda

True

In [18]:
df = pd.read_csv("PYTORCH_NOTEBOOKS/Data/iris.csv")
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [34]:
X = df.drop('target',axis=1).values
y = df.target.values

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=101)

In [36]:
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [37]:
train_loader = DataLoader(dataset=X_train,batch_size=80,shuffle=True,pin_memory=True)
test_loader = DataLoader(dataset=X_test,batch_size=80,shuffle=False,pin_memory=True)

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [39]:
start_time = time.time()

epochs = 100

losses = []

for i in range(epochs):
    y_pred = gpumodel.forward(X_train)
    loss = criterion(y_pred,y_train)
    losses.append(loss)

    if i%10 == 0:
        print(f"Epoch {i} -> Loss: {loss.item()}")
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f"Total time elapsed in model training: {(time.time()-start_time)/60} minutes.")

Epoch 0 -> Loss: 0.27540868520736694
Epoch 10 -> Loss: 0.1764075607061386
Epoch 20 -> Loss: 0.1192159429192543
Epoch 30 -> Loss: 0.08744481205940247
Epoch 40 -> Loss: 0.07043136656284332
Epoch 50 -> Loss: 0.06194848194718361
Epoch 60 -> Loss: 0.0574788935482502
Epoch 70 -> Loss: 0.054719019681215286
Epoch 80 -> Loss: 0.05300268158316612
Epoch 90 -> Loss: 0.05181269720196724
Total time elapsed in model training: 0.004267545541127523 minutes.


In [40]:
correct = 0

with torch.no_grad():
    for idx, data in enumerate(X_test):
        y_val = gpumodel.forward(data)

        if y_val.argmax().item() == y_test[idx]:
            correct += 1

print(f"Accuracy: {correct}/{len(X_test)} = {correct/len(X_test)*100:.2f}%")

Accuracy: 30/30 = 100.00%


In [41]:
torch.cuda.memory_allocated()

17113600